In [ ]:
import pandas as pd
from unidecode import unidecode

In [ ]:
#Importação do df com a classificação dos alimentos
df_classif = pd.read_csv(r'C:\Users\guilh\Documents\Pesquisa\classificacao_alimentos_limpa.csv',usecols=['alimento','classif_NOVA','ultraprocessado','alim_protetores'])

In [ ]:
#Importação do df com os dados dos cardápios
df = pd.read_csv(r'C:\Users\guilh\Documents\Pesquisa\painel_cardapios_com_index_por_cidade.csv',low_memory=False).reset_index()

In [ ]:
#Transformando as colunas com os alimentos em linhas, para que cada alimento do cardápio de determinado dia corresponda a uma observação
df = pd.wide_to_long(df,['Alim'],i='index',j='Alimento').sort_values(by=['Nome_cidade','ano']).reset_index()

In [ ]:
#Retirando espaços e caracteres especiais das colunas correspondentes aos alimentos para facilitar o merge
df['Alim'] = df['Alim'].str.lower().str.replace(" ","").str.replace(",","").str.replace(".","").str.replace("-","").str.replace(":","").apply(lambda s:unidecode(s) if type(s) == str else s)
df_classif['alimento'] = df_classif['alimento'].str.lower().str.replace(" ","").str.replace(",","").str.replace(".","").str.replace("-","").str.replace(":","").apply(lambda s:unidecode(s) if type(s) == str else s)

In [ ]:
#Fazendo o merge dos dfs 
df_vf = pd.merge(df,df_classif,how='left',left_on=['Alim'],right_on=['alimento'])

In [ ]:
#Excluindo as colunas index,Alimento,Unnamed: 0,ID,ID Cidade,Alim,cidades
df_vf.drop(columns=['index','Alimento','Unnamed: 0','ID','ID Cidade','Alim','cidades'],inplace=True)

In [ ]:
#Transformando a coluna Data em formato datetime
df_vf['Data'] = pd.to_datetime(df_vf['Data'],errors='ignore')

In [ ]:
df_vf.sort_values(['Nome_cidade','ano','Data'],inplace=True)

In [ ]:
#Retirando duplicatas
df_vf.drop_duplicates(inplace=True)

In [ ]:
#Setando como index nome da cidade e ano, retirando duplicatas e retirando observações com  NA na coluna alimento 
df_vf = df_vf.set_index(['Nome_cidade','ano']).dropna(subset=['alimento'])

In [ ]:
df_vf.to_csv(r'painel_cardapios_com_classificacao_alimentos.csv')